# Decorators 
## Functions and inner-functions
Lets create a function that prints the fibonnacci sequence 

In [1]:
def fibo(n):
    ''' print fibonacci'''
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2) 

In [2]:
for i in range(10):
    print(f'{i}th fibonacci number is : {fibo(i)}')

0th fibonacci number is : 0
1th fibonacci number is : 1
2th fibonacci number is : 1
3th fibonacci number is : 2
4th fibonacci number is : 3
5th fibonacci number is : 5
6th fibonacci number is : 8
7th fibonacci number is : 13
8th fibonacci number is : 21
9th fibonacci number is : 34


Lets check the type of the `fibo(n)` function

In [74]:
type(fibo)

function

In [75]:
type(fibo(1))

int

Now lets write a function within funtions, this is the basis of using Decorators. Notice that the vapiable passed into the enclosing function `fact_fibo` is local to the inner functions

In [3]:
def fact_fibo(n):
    '''returns the factorial of the nth fibonacci number'''

    def fact(n):
        '''returns the factorial of n'''
        if n == 1 :
            return n
        else:
            return n*fact(n-1)
    
    def fibo(n):
        '''returns the nth fibonacci number'''
        if n<2:
            return n
        else:
            return fibo(n-1) + fibo(n-2)

    return fact(fibo(n))

In [4]:
fact_fibo(6)

40320

## Decorators

Definition: _A decorator is a callable that takes another function as an argument, extending the behaviour of that function without explicitly modifying that function_
* Decorators are particularly useful when applying changes to a large number of function. 
* The original function remains as it is, which relaxes the complexity. 

We now create a wrapper function that takes any interger returing function as input and returns its factorial

In [138]:
# base template:
from functools import wraps

def my_decor(func):
    '''this is a decorator that takes a function object func as input'''
    
    @wraps(func)   #preserving metadata of the decorated function
    def wrapper(*args, **kwargs):  # takes any arguments (to resolve arg conflict with the decorated function)
        '''this is the wrapper function'''
        # do something here
        result = func(*args, **kwargs)  # call the decorated function
        # do something here 
        return result 
    return wrapper    # returning the wrapper function reference out 

@my_decor     #decorating   :    my_func = my_decor(my_func)
def my_func(*args, **kwargs):
    #define the function here 
    pass

In [103]:
from functools import wraps

def wrapper(func):
    '''wraps a function func'''
    @wraps(func)  #alters the metadata of the decorated by the param's 
    def _fibo():
        return 'This is _fibo'
    #_fibo.__name__ = func.__name__
    #_fibo.__doc__ = func.__doc__
    return _fibo

In [99]:
# returns the ref to _fibo() function
wrapper(fibo(5))

<function __main__.wrapper.<locals>._fibo()>

In [80]:
fibo(5)

5

In [81]:
wrapper(fibo(5))

<function __main__.wrapper.<locals>._fibo()>

In [82]:
fibo = wrapper(fibo(5))   #reassigning the function pointer to fibo()

In [83]:
fibo(5)  # the function _fibo has replaced fibo and _fibo() takes 0 argument

TypeError: _fibo() takes 0 positional arguments but 1 was given

In [84]:
fibo()

'This is _fibo'

The decorator simplifies the process by annotating a function with a decorator. 

In [105]:
@wrapper                 
def new_fibo(n):        #    new_fibo = wrapper(new_fibo(n))
        '''returns the nth fibonacci number'''
        if n<2:
            return n
        else:
            return fibo(n-1) + fibo(n-2)

In [86]:
new_fibo()

'This is _fibo'

In [87]:
new_fibo

<function __main__.wrapper.<locals>._fibo()>

# Using @wrap 

*  when we decorate a function the metadata of the decorated function is replaced by the decorating fucntion. 

In [101]:
# check the `__name__` and `__doc_` metadata of the new fibo is replaced by the one of _fibo() 
print(new_fibo.__name__)
print(new_fibo.__doc__)

_fibo
None


 * to solve the issue go to the wrapper function and assign `__name__` and `__doc__` as one of the decorated function's. 
 * Alternatively use the `functools.wrap` module to make  it easier 
 * Do the following 
    1. go to the wrapper function
    2. uncomment the sections 
    3. re rurn the decoraded function's definition 
    4. run the follwoing code

In [107]:
print(new_fibo.__name__)
print(new_fibo.__doc__)

new_fibo
returns the nth fibonacci number


## Challange

Write `bold` and `italic` decorator and apply on a function such that, the the decorators will add correcponsding html syntax to the decorated function. 

In [127]:
# writing the decorators 
def bold(func):

    @wraps(func)       # preserving the metadata
    def inner(text):   # preserving the input parameters 
        return f'<b>{func(text)}</b>'   #decoration 
    return inner  

def italic(func):

    @wraps(func)
    def inner(text):
        return f'<i>{func(text)}</i>'
    return inner  

@bold                   # outer decoration = bold
@italic                 # inner decoration = italic
def test_func(text):     
    '''prints a text given as arg'''
    return(text)

In [128]:
test_func('hello world')

'<b><i>hello world</i></b>'

## Decorators with arguments 

In [133]:
def str_len_sum(*args):
    '''returns the sum of the srings' length entered as argument'''
    sum = 0
    for str in args:
        sum += len(str)
    return sum

In [134]:
str_len_sum()

0

In [135]:
str_len_sum('hello' 'world')

10

In [ ]:
def star(func):
    '''adds start before and after the output of the wrapped function'''
    def wrapper(*args, **kwargs):
        return f'*{func()}*'

## Creating `time_it` decorator to time any function 

In [144]:
def time_it(func):
    '''return execution time of a function func'''
    @wraps(func)
    def wrapper(*args, **kwargs):
        import time
        t = time.time()
        temp = func(*args, **kwargs)
        return time.time() - t
    return wrapper

@time_it
def dummy_sleeper(t):
    '''sleeps for t seconds'''
    import time
    print(f'sleeping for {t} secs....')
    time.sleep(t)
    print('done...')

In [145]:
dummy_sleeper(3)

sleeping for 3 secs....
done...


3.014361619949341

## Challange

Write a `@munch` decorator that takes a function with 2 integers, _start_(inclusive) and _end_(exclusive). The decorator is applicable to any string returning function and munches the substring from _start_ to _end_ with 'X' 

In [164]:
def munch(start, end,ch):
    def do_munch(func):
        ''' munches substring from start to end by replcing char with ch'''
        @wraps(func)
        def wrapper(*args, **kwargs):
            ret =''
            res = func(*args, **kwargs)
            for i in range(len(res)):
                if i >= start and i<end and res[i] != ' ':
                    ret+='X'
                else:
                    ret+=res[i]
            return ret
        return wrapper
    return do_munch

@munch(1,10,'x')
def test_func(s):
    '''returns the input string s'''
    return s

In [165]:
test_func('this is a test string')

'tXXX XX X test string'